# Web Scraping For Fun!

## Step 1: Download College directory listing using Selenium and Chrome

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# Initialize selenium chrome web driver and URL variables
driver = webdriver.Chrome()

# A funtion to select correct options and download data
def download(url):
    # Passing URL to browser
    driver.get(url)
    # selecting EXCEL from dropdown list 
    select = Select(driver.find_element_by_name("reportFormatType"))
    select.select_by_value('2')
    # Selecting the "search button"
    button = driver.find_element_by_class_name("btn-primary")
    button.click()
    
url = f"http://aishe.nic.in/aishe/collegeDirectoryIndex?hasReportLink=index"
download(url)

## Step 2: Read and list website URL into a different file.
Expecting all three files downloaded earlier to be there in the uni_data folder.

In [31]:
import os
(_, _, filenames) = next(os.walk("uni_data"))
import pandas as pd
# loading the file to a pandas dataframe
df = pd.read_excel(f"uni_data/{filenames[0]}",header=2,skiprows=[3,3])
df = df[['College Name','Address','Website']]

In [32]:
# renaming columns
df.rename(columns={'College Name':'name','Address':'address','Website':'website'},inplace=True)
df.head(5)

,name,address,website
0,Jawaharlal Nehru Rajkeeya Mahavidyalaya (Id: ...,"Jawaharlal Nehru Rajkeeya Mahavidyalaya, M S ...",jnrm.and.nic.in
1,Regional Medical Research Institute (I.C.M.R.)...,"Post Bag No 13, Dollygunj, Port Blair",http://www.rmrc.res.in
2,Tagore Government College of Education (Id: C-...,"M.G. Road, Middle Point, Port Blair - 744101",www.tgce.and.nic.in
3,Zoological Survey of India (Id: C-6560),"Zoological Survey of India, Andaman & Nicobar ...",www.zsi.gov.in
4,Mahatma Gandhi Govt. College (Id: C-6547),"Rampur, Mayabunder, Mayabunder",http://mggcm.and.nic.in


In [56]:
# adding new columns to df-copy
dfc = df.copy()
#clean url from df
def clean_url(url):
    if isinstance(url, str): 
        if (url.startswith("http://") or url.startswith("https://")):
            return url
        else:
            return "http://"+url
    else:
        return 'nan'
# cleaning URL's
dfc["website"] = dfc.apply(lambda row : clean_url(row['website']), axis=1)
dfc.to_csv("uni_data/cleanedOriginal.csv")

## Step 3: Loop through URLs and scrap data.
#### Add Email, Contact No to the df

In [115]:
# imp package imports
import itertools
# possible url extensions
dir_lv1 = ['/Contact/','/Contact-us/','/ContactUs/','/Contact_us/','/']
dir_lv2 = ['contact','contact_us','contact-us','contactus']
# possible file extensions
str1 = ".ARO .ASHX .ASP .ASPX .ATOM .ATT .AXD .CHM .CMS .CSS .DCR .DHTML .DISCOMAP .DOWNLOAD .ECE .ECE .EPIBRW .GNE .HTM .HTML .HTMLS .HXS .JCZ .JNLP .JS .JSON .JSP .JSPX .JVS .JWS .LASSO .MASTER .MSPX .NXG .OGNC .PHP .PHP3 .QRM .RHTML .RMD .RSS .RWP .SAVEDDECK .SVC .UCF .URL .WEBLOC .WSDL .XHTML .XPD .ZFO .ZUL"
formats_i = str1.split()
formats_i.extend([''])
# list all url combinations
def combinator(l1, l2, l3):
    comb_0=[]
    for i in l1:
        for j in l2:
            for k in l3:
                comb_0.append(i+j+k)
    return comb_0
T1_comb = combinator(dir_lv1, dir_lv2, formats_i)
contactno = []
emailid = []

In [116]:
# scrape webpage and extract email ID
def emailscrap(url, comb):
    el1 = []
    for i in comb:
        url = url+i
        print(url)
        try:
            response = requests.get(url)
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
            # ignore pages with errors and continue with next url
            continue
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.com", response.text, re.I)) # re.I: (ignore case)
        el1.extend(new_emails)
    return " ".join(el1)

In [117]:
emailscrap("http://jnrm.and.nic.in", T1_comb)

http://jnrm.and.nic.in/Contact/contact.ARO


'nan'

In [110]:
dft = dfc.copy()
dft["emails"] = dft.apply(lambda row : emailscrap(row['website'], T1_comb), axis=1)

In [111]:
dft

,name,address,website,emails
0,Jawaharlal Nehru Rajkeeya Mahavidyalaya (Id: ...,"Jawaharlal Nehru Rajkeeya Mahavidyalaya, M S ...",http://jnrm.and.nic.in,
1,Regional Medical Research Institute (I.C.M.R.)...,"Post Bag No 13, Dollygunj, Port Blair",http://www.rmrc.res.in,
2,Tagore Government College of Education (Id: C-...,"M.G. Road, Middle Point, Port Blair - 744101",http://www.tgce.and.nic.in,
3,Zoological Survey of India (Id: C-6560),"Zoological Survey of India, Andaman & Nicobar ...",http://www.zsi.gov.in,
4,Mahatma Gandhi Govt. College (Id: C-6547),"Rampur, Mayabunder, Mayabunder",http://mggcm.and.nic.in,
...,...,...,...,...
44398,Ramkrishna Mahato Memorial Teachers Training C...,"VILLAGE- MANAKCHITA, POST OFFICE- BARAGRAM, PO...",http://www.rmmttcollege.org.in,
44399,SIMANTA MANBHUM TEACHERS TRAINING COLLEGE (Id:...,"Chelyama, Raghunathpur",http://www.smttchelyama.org,
44400,Sponsored Teachers' Training college (Id: C-44...,"Deshbandhu Road, P.O : Purulia, Purulia",http://www.sttcollege.org,
44401,Vidyasagar Foundation School of Education & Tr...,"Vill - Tamna, POST - SIMULIA , PIN- 723102, PU...",http://www.vidyasagarfoundation.org.in,
